In [19]:
import pandas as pd
import pathlib

# Treinamento e Teste

- 90% para Treinamento;
- 10% para Testes;

In [20]:
mgd = 'raw'
df = pd.read_csv(f'../gtzan/processed/mgd_{mgd}.csv')
group_by_label = df.groupby('label')
seed = 0

In [21]:
df_test = group_by_label.sample(frac=0.1,
                                random_state=seed)
df_train = df[~df['song_id'].isin(df_test['song_id'])]

In [22]:
# Checagem: músicas de teste não devem estar no treino (e vice-versa)
assert df_test['song_id'].isin(df_train['song_id']).any() == False
assert df_train['song_id'].isin(df_test['song_id']).any() == False

In [23]:
save_path = pathlib.Path('../gtzan/processed/').joinpath(mgd)

if not save_path.exists():
    save_path.mkdir(parents=True,
                    exist_ok=False)

df_train.to_csv(save_path.joinpath('train.csv'), index=False)
df_test.to_csv(save_path.joinpath('test.csv'), index=False)

# Criação dos 10-folds para 5 experimentos

In [24]:
# Seed para geração dos folds para cada experimento
folds_seeds = [0, 1, 2, 3, 4]
assert len(folds_seeds) == 5

for i in range(1, 6):
    # Shuffle
    rand_df = df_train.sample(frac=1,
                              random_state=folds_seeds[i-1])

    # O dataset de treino deve possuir 900 músicas
    assert len(rand_df) == 900

    exp_save_path = save_path.joinpath(f'experiment-{i}')

    if not exp_save_path.exists():
        exp_save_path.mkdir(parents=True)

    # 10 folds por experimento,
    for j in range(0, 10):
        l = j * 90
        u = l + 90

        df_fold = rand_df.iloc[l:u, :]
        fold_save_path = exp_save_path.joinpath(f'fold-{j+1}.csv')

        # Cada fold deve possuir 90 músicas
        assert len(df_fold) == 90

        df_fold.to_csv(fold_save_path, index=False)